In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
gender_submission = pd.read_csv('gender_submission.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
gender_submission = pd.read_csv('gender_submission.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train = train.drop('Name', axis=1)
test = test.drop('Name', axis=1)
train_target = train['Survived']
train = train.drop('Survived', axis=1)

train['Embarked'] = train['Embarked'].fillna('S')
train['Cabin'] = train['Cabin'].fillna('0')
train['Age'] = train['Age'].fillna(0)
train['Sex'] = train['Sex'].apply(lambda x: 1 if x =='male' else 2)
test['Embarked'] = test['Embarked'].fillna('S')
test['Cabin'] = test['Cabin'].fillna('0')
test['Age'] = test['Age'].fillna(0)
test['Sex'] = test['Sex'].apply(lambda x: 1 if x =='male' else 2)


def get_ticket(data):
    res = []
    for i in np.arange(data.shape[0]):
        if len(data[i].split()) == 1:
            res.append('0')
        else:
            res.append(data[i].split()[0])
    return res

train['Ticket'] = get_ticket(train['Ticket'])
test['Ticket'] = get_ticket(test['Ticket'])


from sklearn.preprocessing import LabelEncoder

train['Ticket'] = LabelEncoder().fit_transform(train['Ticket'])
train['Cabin'] = LabelEncoder().fit_transform(train['Cabin'])
train['Embarked'] = LabelEncoder().fit_transform(train['Embarked'])
test['Ticket'] = LabelEncoder().fit_transform(test['Ticket'])
test['Cabin'] = LabelEncoder().fit_transform(test['Cabin'])
test['Embarked'] = LabelEncoder().fit_transform(test['Embarked'])


X_age = train[train['Age'] != 0].drop('Age', axis=1)
y_age = train[train['Age'] != 0]['Age']

from sklearn.ensemble import GradientBoostingRegressor

booster = GradientBoostingRegressor()
booster.fit(X_age, y_age)
y_age_pred = booster.predict(train[train['Age'] == 0].drop('Age', axis=1))

def get_age(data):
    for i in np.arange(data.shape[0]):
        if data['Age'].iloc[i] == 0:
            data['Age'].iloc[i] = booster.predict([data.iloc[0].drop('Age').values])
    return data

train = get_age(train)
test = get_age(test)

c:\Users\AubakirovMA\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
C:\Users\AubakirovMA\AppData\Local\Temp\ipykernel_15100\1330659585.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Age'].iloc[i] = booster.predict([data.iloc[0].drop('Age').values])
c:\Users\AubakirovMA\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
C:\Users\AubakirovMA\AppData\Local\Temp\ipykernel_15100\1330659585.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documenta

In [4]:
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())

In [5]:
X = train
y = train_target

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [13]:
import xgboost as xgb

# параметры оптимизированы optun'ой
params = {'learning_rate': 0.06329973864656831,
 'max_depth': 6,
 'subsample': 0.9814063371832862,
 'colsample_bytree': 0.41087811860602663,
 'min_child_weight': 8}

model_xgb = xgb.XGBClassifier()
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [35]:
# Create XGB Classifier object
xgb_clf = xgb.XGBClassifier()

# Create parameter grid
parameters = {"learning_rate": [0.1, 0.01, 0.05, 0.007, 0.001],
               "gamma" : [0.01, 0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 1],
               "max_depth": [2, 4, 7, 10, 13, 15],
               "colsample_bytree": [0.3, 0.6, 0.8, 1.0],
               "subsample": [0.2, 0.4, 0.5, 0.6, 0.7],
               "reg_alpha": [0, 0.5, 1],
               "reg_lambda": [1, 1.5, 2, 3, 4.5],
               "min_child_weight": [1, 3, 5, 7],
               "n_estimators": [100, 250, 500, 1000, 1200, 1500]}

from sklearn.model_selection import RandomizedSearchCV
# Create RandomizedSearchCV Object
xgb_rscv = RandomizedSearchCV(xgb_clf, param_distributions = parameters, scoring = "f1_micro",
                             cv = 10, verbose = -1, random_state = 40 )

# Fit the model
model_xgboost = xgb_rscv.fit(X, y)

# Model best estimators
print("Learning Rate: ", model_xgboost.best_estimator_.get_params()["learning_rate"])
print("Gamma: ", model_xgboost.best_estimator_.get_params()["gamma"])
print("Max Depth: ", model_xgboost.best_estimator_.get_params()["max_depth"])
print("Subsample: ", model_xgboost.best_estimator_.get_params()["subsample"])
print("Max Features at Split: ", model_xgboost.best_estimator_.get_params()["colsample_bytree"])
print("Alpha: ", model_xgboost.best_estimator_.get_params()["reg_alpha"])
print("Lamda: ", model_xgboost.best_estimator_.get_params()["reg_lambda"])
print("Minimum Sum of the Instance Weight Hessian to Make a Child: ",
      model_xgboost.best_estimator_.get_params()["min_child_weight"])
print("Number of Trees: ", model_xgboost.best_estimator_.get_params()["n_estimators"])

Learning Rate:  0.01
Gamma:  0.25
Max Depth:  13
Subsample:  0.5
Max Features at Split:  0.6
Alpha:  1
Lamda:  3
Minimum Sum of the Instance Weight Hessian to Make a Child:  3
Number of Trees:  1500


In [36]:
from sklearn.metrics import classification_report


y_pred = model_xgboost.predict(X_train)
y_pred_test = model_xgboost.predict(X_test)

print(classification_report(y_train, y_pred))
print()
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.88      0.96      0.92       444
           1       0.92      0.79      0.85       268

    accuracy                           0.89       712
   macro avg       0.90      0.87      0.88       712
weighted avg       0.90      0.89      0.89       712


              precision    recall  f1-score   support

           0       0.87      0.91      0.89       105
           1       0.87      0.81      0.84        74

    accuracy                           0.87       179
   macro avg       0.87      0.86      0.87       179
weighted avg       0.87      0.87      0.87       179



In [37]:
# 0.77272
# 3791 / 14931

gender_submission['Survived'] = model_xgboost.predict(test)
gender_submission.to_csv('submission.csv', index=False)
pd.read_csv('submission.csv')

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [175]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

#Создаём объект класса RandomForestClassifier
rf_clf_2d = RandomForestClassifier(
    n_estimators=500, #число деревьев
    criterion='entropy', #критерий эффективности
    max_depth=15, #максимальная глубина дерева
    max_features='sqrt', #число признаков из метода случайных подпространств
    random_state=42 #генератор случайных чисел
)
#Обучаем модель 
rf_clf_2d.fit(X, y)
 
#Делаем предсказание класса
y_pred = rf_clf_2d.predict(X)
#Выводим отчёт о метриках
print(classification_report(y, y_pred))



              precision    recall  f1-score   support

           0       0.98      1.00      0.99       549
           1       1.00      0.97      0.99       342

    accuracy                           0.99       891
   macro avg       0.99      0.99      0.99       891
weighted avg       0.99      0.99      0.99       891



In [180]:
test_target = rf_clf_2d.predict(test)
test_target


array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [182]:
# 0.77751
# 3791 / 14931

gender_submission['Survived'] = test_target
gender_submission.to_csv('submission.csv', index=False)
pd.read_csv('submission.csv')

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


*** 
# Stop

In [122]:
train.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [123]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [124]:
train['Embarked'] = train['Embarked'].fillna('S')
train['Cabin'] = train['Cabin'].fillna('0')
train['Age'] = train['Age'].fillna(0)
train['Sex'] = train['Sex'].apply(lambda x: 1 if x =='male' else 2)

In [125]:
def get_ticket(data):
    res = []
    for i in np.arange(data.shape[0]):
        if len(data[i].split()) == 1:
            res.append('0')
        else:
            res.append(data[i].split()[0])
    return res

train['Ticket'] = get_ticket(train['Ticket'])

In [126]:
train.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5,7.2500,0,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",2,38.0,1,0,PC,71.2833,C85,C


In [127]:
from sklearn.preprocessing import LabelEncoder

train['Ticket'] = LabelEncoder().fit_transform(train['Ticket'])
train['Cabin'] = LabelEncoder().fit_transform(train['Cabin'])
train['Embarked'] = LabelEncoder().fit_transform(train['Embarked'])

In [128]:
train = train.drop('Name', axis=1)

In [129]:
X_age = train[train['Age'] != 0].drop('Age', axis=1)
y_age = train[train['Age'] != 0]['Age']

In [133]:
from sklearn.ensemble import GradientBoostingRegressor

booster = GradientBoostingRegressor()
booster.fit(X_age, y_age)
y_age_pred = booster.predict(train[train['Age'] == 0].drop('Age', axis=1))

In [155]:
def get_age(data):
    for i in np.arange(data.shape[0]):
        if data['Age'].iloc[i] == 0:
            data['Age'].iloc[i] = booster.predict([data.iloc[0].drop('Age').values])
    return data

train = get_age(train)

In [158]:
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [157]:
train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,1,22.000000,1,0,5,7.2500,0,2
1,2,1,1,2,38.000000,1,0,18,71.2833,82,0
2,3,1,3,2,26.000000,0,0,37,7.9250,0,2
3,4,1,1,2,35.000000,1,0,0,53.1000,56,2
4,5,0,3,1,35.000000,0,0,0,8.0500,0,2
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,1,27.000000,0,0,0,13.0000,0,2
887,888,1,1,2,19.000000,0,0,0,30.0000,31,2
888,889,0,3,2,27.927682,1,2,39,23.4500,0,2
889,890,1,1,1,26.000000,0,0,0,30.0000,61,0
